In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier,plot_importance,DMatrix,cv
import xgboost as xgb
from sklearn.metrics import f1_score,recall_score,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_auc_score, roc_curve, auc
import warnings, gc
warnings.filterwarnings("ignore")
import plotly.express as px
import plotly.graph_objects as go
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
import tensorflow as tf
from sklearn.cluster import KMeans 
from scipy.spatial.distance import cdist 
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder

from tqdm import tqdm

In [31]:
train_df = pd.read_parquet('train.parquet',engine='pyarrow')
test_df = pd.read_parquet('test.parquet',engine='pyarrow')

In [32]:
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
cat_features_light = ["B_30_last","B_38_last","D_114_last","D_116_last","D_117_last","D_120_last","D_126_last","D_63_last","D_64_last","D_66_last","D_68_last",
                     "B_30_last_2","B_38_last_2","D_114_last_2","D_116_last_2","D_117_last_2","D_120_last_2","D_126_last_2","D_63_last_2","D_64_last_2","D_66_last_2","D_68_last_2",
                     "B_30_last_3","B_38_last_3","D_114_last_3","D_116_last_3","D_117_last_3","D_120_last_3","D_126_last_3","D_63_last_3","D_64_last_3","D_66_last_3","D_68_last_3"]
num_cols = [x for x in train_df.columns if (x not in cat_features_light) & (x not in ['customer_ID','S_2','target'])]

In [33]:
print('train的特徵數:',len(train_df.columns))
print('test的特徵數:',len(test_df.columns))
print('train的最小值:',train_df.min().min())
print('test的最小值:',test_df.min().min())

train的特徵數: 2213
test的特徵數: 2212
train的最小值: -9103.230091108475
test的最小值: -31679.20527486957


In [34]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64']
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if (str(col_type)[:3] == 'int') and (df[col].isna().sum() !=0):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
    return df

In [35]:
train_feature = ['train_meta_ad','train_meta_cat','train_meta_cat_lv2','train_meta_et','train_meta_kmean_2','train_meta_kmean_3','train_meta_knn_2','train_meta_knn_4','train_meta_knn_8','train_meta_knn_16','train_meta_knn_32','train_meta_knn_64','train_meta_knn_128','train_meta_knn_256','train_meta_knn_512','train_meta_knn_1024','train_meta_light','train_meta_light_lv2','train_meta_rf','train_meta_rf_lv2','train_meta_xg','train_meta_xg_lv2','train_meta_nnt']
test_feature = ['test_meta_ad','test_meta_cat','test_meta_cat_lv2','test_meta_et','test_meta_kmean_2','test_meta_kmean_3','test_meta_knn_2','test_meta_knn_4','test_meta_knn_8','test_meta_knn_16','test_meta_knn_32','test_meta_knn_64','test_meta_knn_128','test_meta_knn_256','test_meta_knn_512','test_meta_knn_1024','test_meta_light','test_meta_light_lv2','test_meta_rf','test_meta_rf_lv2','test_meta_xg','test_meta_xg_lv2','test_meta_nnt']

In [36]:
# for i in train_feature:
#     _ = pd.read_parquet(f'{i}.parquet')
#     print(_.columns,i)

In [37]:
# train_df['pred_nn_lv2']

In [38]:
len(train_feature),len(test_feature)

(23, 23)

In [39]:
target = pd.read_parquet('label.parquet')
target = target.set_index('customer_ID',drop=True)

In [40]:
# train_df = pd.DataFrame()
for feature in tqdm(train_feature):
    _ = pd.read_parquet(f'{feature}.parquet')
    train_df = pd.concat([train_df,_],axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:53<00:00,  2.32s/it]


In [41]:
# test_df = pd.DataFrame()
for feature in tqdm(test_feature):
    _ = pd.read_parquet(f'{feature}.parquet')
    test_df = pd.concat([test_df,_],axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [02:11<00:00,  5.71s/it]


In [42]:
# train_df = pd.concat([train_df,target],axis=1)

In [43]:
test_train = train_df.drop('target',axis=1)
for i in tqdm(range(len(test_train.columns))):
    if test_train.columns[i] != test_df.columns[i]:
        print(test_train.columns[i] ,"miss match",test_df.columns[i])
del test_train

100%|█████████████████████████████████████████████████████████████████████████| 2235/2235 [00:00<00:00, 1121189.98it/s]


In [44]:
train_df.info(),test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 458913 entries, 116a83161af5342b73d35c9cd9dc7d7552a85bf7c8d7735c28eb9ef08a77f7ff to 6390c75e546a04e755b300070e987715e40dfec3e73c35893a5c3403616d24e4
Columns: 2236 entries, P_2_mean to pred_nn_lv2
dtypes: float32(561), float64(1008), int16(27), int32(2), int64(402), int8(236)
memory usage: 5.9+ GB
<class 'pandas.core.frame.DataFrame'>
Index: 924621 entries, 00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7 to fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61cceb803ea8ec37634d
Columns: 2235 entries, P_2_mean to pred_nn_lv2
dtypes: float32(561), float64(997), int16(30), int64(414), int8(233)
memory usage: 11.9+ GB


(None, None)

In [45]:
train_df = train_df.fillna(-9999)
test_df = test_df.fillna(-9999)

# model

In [46]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)


def plot_roc(y_val,y_prob):
    colors=px.colors.qualitative.Prism
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=np.linspace(0,1,11), y=np.linspace(0,1,11), 
                             name='Random Chance',mode='lines', showlegend=False,
                             line=dict(color="Black", width=1, dash="dot")))
    for i in range(len(y_val)):
        y=y_val[i]
        prob=y_prob[i]
        fpr, tpr, _ = roc_curve(y, prob)
        roc_auc = auc(fpr,tpr)
        fig.add_trace(go.Scatter(x=fpr, y=tpr, line=dict(color=colors[::-1][i+1], width=3), 
                                 hovertemplate = 'True positive rate = %{y:.3f}<br>False positive rate = %{x:.3f}',
                                 name='Fold {}:  Gini = {:.3f}, AUC = {:.3f}'.format(i+1, gini[i],roc_auc)))
    fig.update_layout(template=temp, title="Cross-Validation ROC Curves", 
                      hovermode="x unified", width=700,height=600,
                      xaxis_title='False Positive Rate (1 - Specificity)',
                      yaxis_title='True Positive Rate (Sensitivity)',
                      legend=dict(orientation='v', y=.07, x=1, xanchor="right",
                                  bordercolor="black", borderwidth=.5))
    fig.show()

In [47]:
X=train_df.drop(['target'],axis=1)
y=train_df['target']
y_valid, gbm_val_probs, gbm_test_preds, gini,auc_score_list=[],[],[],[],[]
ft_importance=pd.DataFrame(index=X.columns)
sk_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=21)
for fold, (train_idx, val_idx) in enumerate(sk_fold.split(X, y)):
    
    print("\nFold {}".format(fold+1))
    X_train, y_train = X.iloc[train_idx,:], y[train_idx]
    X_val, y_val = X.iloc[val_idx,:], y[val_idx]
    print("Train shape: {}, {}, Valid shape: {}, {}\n".format(
        X_train.shape, y_train.shape, X_val.shape, y_val.shape))
##原始參數    
#     params = {'boosting_type': 'gbdt',
#               'n_estimators': 1000,
#               'num_leaves': 50,
#               'learning_rate': 0.05,
#               'colsample_bytree': 0.9,
#               'min_child_samples': 2000,
#               'max_bins': 500,
#               'reg_alpha': 2,
#               'objective': 'binary',
#               'random_state': 21}

#     params = {
#         'objective': 'binary',
#         'n_estimators': 1500,
#         'metric': "binary_logloss",
#         'boosting': 'gbdt',    #dart
#         'seed': 21,    
#         'num_leaves': 100,
#         'learning_rate': 0.05,
#         'feature_fraction': 0.20,
#         'bagging_freq': 10,
#         'bagging_fraction': 0.50,
#         'lambda_l2': 2,
#         'min_data_in_leaf': 40
#         }
    #seed try 21 跟42
    #42 is wate you need
    
    params = {'n_estimators':1200,'learning_rate':0.03, 'reg_lambda':50,'min_child_samples':2400,'num_leaves':95,
              'colsample_bytree':0.19,'max_bins':511, 'random_state':1}

#     lgbm_params ={"objective":"binary",
#               'n_estimators':1200,
#           'num_leaves': 69,
#             'boosting':'dart',
#           'max_bin': 312,
#           'feature_fraction': 0.5193052325515839,
#           'bagging_fraction': 0.597903764208054,
#           'bagging_freq': 0,
#           'min_data_in_leaf': 21,
#           "random_seed":42}
    
    
    
    
    gbm = LGBMClassifier(**params).fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)],
                                       eval_metric=['auc','binary_logloss'])
    
    gbm_prob = gbm.predict_proba(X_val)[:,1]
    gbm_val_probs.append(gbm_prob)
    y_valid.append(y_val)
    
    y_pred=pd.DataFrame(data={'prediction':gbm_prob})
    y_true=pd.DataFrame(data={'target':y_val.reset_index(drop=True)})
    gini_score=amex_metric(y_true = y_true, y_pred = y_pred)
    gini.append(gini_score)
    
    auc_score=roc_auc_score(y_val, gbm_prob)
    auc_score_list.append(auc_score)
    gbm_test_preds.append(gbm.predict_proba(test_df)[:,1])    
    ft_importance["Importance_Fold"+str(fold)]=gbm.feature_importances_    
    print("Validation Gini: {:.5f}, AUC: {:.4f}".format(gini_score,auc_score))
    
    del X_train, y_train, X_val, y_val
    _ = gc.collect()
    
del X, y


Fold 1
Train shape: (413021, 2235), (413021,), Valid shape: (45892, 2235), (45892,)

[1]	training's auc: 0.958916	training's binary_logloss: 0.552968	valid_1's auc: 0.95791	valid_1's binary_logloss: 0.552986
[2]	training's auc: 0.960448	training's binary_logloss: 0.535486	valid_1's auc: 0.959523	valid_1's binary_logloss: 0.535522
[3]	training's auc: 0.960575	training's binary_logloss: 0.519356	valid_1's auc: 0.959677	valid_1's binary_logloss: 0.519412
[4]	training's auc: 0.960661	training's binary_logloss: 0.504409	valid_1's auc: 0.959749	valid_1's binary_logloss: 0.504481
[5]	training's auc: 0.961018	training's binary_logloss: 0.49049	valid_1's auc: 0.960232	valid_1's binary_logloss: 0.490584
[6]	training's auc: 0.961042	training's binary_logloss: 0.477492	valid_1's auc: 0.960258	valid_1's binary_logloss: 0.477602
[7]	training's auc: 0.961297	training's binary_logloss: 0.465315	valid_1's auc: 0.960525	valid_1's binary_logloss: 0.465446
[8]	training's auc: 0.961335	training's binary_l

[68]	training's auc: 0.962585	training's binary_logloss: 0.23688	valid_1's auc: 0.961526	valid_1's binary_logloss: 0.238232
[69]	training's auc: 0.962595	training's binary_logloss: 0.236031	valid_1's auc: 0.96153	valid_1's binary_logloss: 0.237412
[70]	training's auc: 0.962603	training's binary_logloss: 0.235219	valid_1's auc: 0.961535	valid_1's binary_logloss: 0.236618
[71]	training's auc: 0.962612	training's binary_logloss: 0.234433	valid_1's auc: 0.961535	valid_1's binary_logloss: 0.235865
[72]	training's auc: 0.962629	training's binary_logloss: 0.233727	valid_1's auc: 0.961532	valid_1's binary_logloss: 0.235186
[73]	training's auc: 0.96264	training's binary_logloss: 0.233	valid_1's auc: 0.961543	valid_1's binary_logloss: 0.234485
[74]	training's auc: 0.962653	training's binary_logloss: 0.232291	valid_1's auc: 0.961547	valid_1's binary_logloss: 0.23381
[75]	training's auc: 0.962686	training's binary_logloss: 0.231605	valid_1's auc: 0.961559	valid_1's binary_logloss: 0.233163
[76]	tr

[134]	training's auc: 0.963529	training's binary_logloss: 0.214391	valid_1's auc: 0.961638	valid_1's binary_logloss: 0.218537
[135]	training's auc: 0.96355	training's binary_logloss: 0.214275	valid_1's auc: 0.961639	valid_1's binary_logloss: 0.218479
[136]	training's auc: 0.963565	training's binary_logloss: 0.214166	valid_1's auc: 0.961639	valid_1's binary_logloss: 0.218412
[137]	training's auc: 0.963581	training's binary_logloss: 0.21406	valid_1's auc: 0.961642	valid_1's binary_logloss: 0.218356
[138]	training's auc: 0.963598	training's binary_logloss: 0.213955	valid_1's auc: 0.961642	valid_1's binary_logloss: 0.218303
[139]	training's auc: 0.963613	training's binary_logloss: 0.213854	valid_1's auc: 0.961638	valid_1's binary_logloss: 0.218251
[140]	training's auc: 0.963628	training's binary_logloss: 0.213753	valid_1's auc: 0.961636	valid_1's binary_logloss: 0.218203
[141]	training's auc: 0.96365	training's binary_logloss: 0.213652	valid_1's auc: 0.961634	valid_1's binary_logloss: 0.21

[201]	training's auc: 0.964772	training's binary_logloss: 0.209413	valid_1's auc: 0.961628	valid_1's binary_logloss: 0.217272
[202]	training's auc: 0.96479	training's binary_logloss: 0.209357	valid_1's auc: 0.961626	valid_1's binary_logloss: 0.217273
[203]	training's auc: 0.964811	training's binary_logloss: 0.20929	valid_1's auc: 0.961626	valid_1's binary_logloss: 0.217271
[204]	training's auc: 0.964831	training's binary_logloss: 0.209229	valid_1's auc: 0.961622	valid_1's binary_logloss: 0.217272
[205]	training's auc: 0.964853	training's binary_logloss: 0.209165	valid_1's auc: 0.961622	valid_1's binary_logloss: 0.217266
[206]	training's auc: 0.964872	training's binary_logloss: 0.209106	valid_1's auc: 0.961622	valid_1's binary_logloss: 0.217263
[207]	training's auc: 0.964891	training's binary_logloss: 0.209047	valid_1's auc: 0.961624	valid_1's binary_logloss: 0.217258
[208]	training's auc: 0.964909	training's binary_logloss: 0.208988	valid_1's auc: 0.961624	valid_1's binary_logloss: 0.2

[267]	training's auc: 0.966127	training's binary_logloss: 0.205405	valid_1's auc: 0.961555	valid_1's binary_logloss: 0.217432
[268]	training's auc: 0.96615	training's binary_logloss: 0.205342	valid_1's auc: 0.961554	valid_1's binary_logloss: 0.217431
[269]	training's auc: 0.966172	training's binary_logloss: 0.205277	valid_1's auc: 0.961552	valid_1's binary_logloss: 0.217434
[270]	training's auc: 0.966194	training's binary_logloss: 0.205214	valid_1's auc: 0.961547	valid_1's binary_logloss: 0.217444
[271]	training's auc: 0.966215	training's binary_logloss: 0.205153	valid_1's auc: 0.961545	valid_1's binary_logloss: 0.217447
[272]	training's auc: 0.966233	training's binary_logloss: 0.205101	valid_1's auc: 0.961545	valid_1's binary_logloss: 0.217447
[273]	training's auc: 0.966254	training's binary_logloss: 0.205042	valid_1's auc: 0.961545	valid_1's binary_logloss: 0.217447
[274]	training's auc: 0.966275	training's binary_logloss: 0.204981	valid_1's auc: 0.961542	valid_1's binary_logloss: 0.

...

[933]	training's auc: 0.978568	training's binary_logloss: 0.169864	valid_1's auc: 0.962985	valid_1's binary_logloss: 0.213469
[934]	training's auc: 0.978587	training's binary_logloss: 0.169813	valid_1's auc: 0.962986	valid_1's binary_logloss: 0.213467
[935]	training's auc: 0.978603	training's binary_logloss: 0.169763	valid_1's auc: 0.962985	valid_1's binary_logloss: 0.21347
[936]	training's auc: 0.978621	training's binary_logloss: 0.169712	valid_1's auc: 0.962984	valid_1's binary_logloss: 0.213472
[937]	training's auc: 0.978635	training's binary_logloss: 0.169664	valid_1's auc: 0.96298	valid_1's binary_logloss: 0.213485
[938]	training's auc: 0.978652	training's binary_logloss: 0.169616	valid_1's auc: 0.962977	valid_1's binary_logloss: 0.213492
[939]	training's auc: 0.978667	training's binary_logloss: 0.169571	valid_1's auc: 0.962979	valid_1's binary_logloss: 0.213484
[940]	training's auc: 0.978684	training's binary_logloss: 0.169524	valid_1's auc: 0.962974	valid_1's binary_logloss: 0.2

[999]	training's auc: 0.979619	training's binary_logloss: 0.166741	valid_1's auc: 0.962925	valid_1's binary_logloss: 0.213637
[1000]	training's auc: 0.979634	training's binary_logloss: 0.166691	valid_1's auc: 0.962923	valid_1's binary_logloss: 0.213642
[1001]	training's auc: 0.979648	training's binary_logloss: 0.16665	valid_1's auc: 0.962925	valid_1's binary_logloss: 0.213639
[1002]	training's auc: 0.979663	training's binary_logloss: 0.166605	valid_1's auc: 0.962919	valid_1's binary_logloss: 0.213653
[1003]	training's auc: 0.979679	training's binary_logloss: 0.166557	valid_1's auc: 0.962922	valid_1's binary_logloss: 0.213648
[1004]	training's auc: 0.979696	training's binary_logloss: 0.166507	valid_1's auc: 0.962921	valid_1's binary_logloss: 0.213653
[1005]	training's auc: 0.979711	training's binary_logloss: 0.166463	valid_1's auc: 0.962921	valid_1's binary_logloss: 0.213655
[1006]	training's auc: 0.979725	training's binary_logloss: 0.16642	valid_1's auc: 0.962923	valid_1's binary_loglo

[1064]	training's auc: 0.980595	training's binary_logloss: 0.163798	valid_1's auc: 0.962909	valid_1's binary_logloss: 0.213674
[1065]	training's auc: 0.980609	training's binary_logloss: 0.163752	valid_1's auc: 0.962911	valid_1's binary_logloss: 0.213667
[1066]	training's auc: 0.980623	training's binary_logloss: 0.163712	valid_1's auc: 0.962911	valid_1's binary_logloss: 0.213665
[1067]	training's auc: 0.980639	training's binary_logloss: 0.163664	valid_1's auc: 0.962912	valid_1's binary_logloss: 0.213662
[1068]	training's auc: 0.980653	training's binary_logloss: 0.163617	valid_1's auc: 0.962911	valid_1's binary_logloss: 0.213667
[1069]	training's auc: 0.98067	training's binary_logloss: 0.163571	valid_1's auc: 0.96291	valid_1's binary_logloss: 0.213667
[1070]	training's auc: 0.980685	training's binary_logloss: 0.163524	valid_1's auc: 0.962908	valid_1's binary_logloss: 0.213671
[1071]	training's auc: 0.980698	training's binary_logloss: 0.163484	valid_1's auc: 0.962907	valid_1's binary_logl

[1129]	training's auc: 0.981527	training's binary_logloss: 0.160935	valid_1's auc: 0.962853	valid_1's binary_logloss: 0.21382
[1130]	training's auc: 0.981542	training's binary_logloss: 0.160889	valid_1's auc: 0.962854	valid_1's binary_logloss: 0.213818
[1131]	training's auc: 0.981556	training's binary_logloss: 0.160845	valid_1's auc: 0.962854	valid_1's binary_logloss: 0.213817
[1132]	training's auc: 0.981571	training's binary_logloss: 0.160801	valid_1's auc: 0.962855	valid_1's binary_logloss: 0.213816
[1133]	training's auc: 0.981585	training's binary_logloss: 0.160758	valid_1's auc: 0.962857	valid_1's binary_logloss: 0.213811
[1134]	training's auc: 0.981598	training's binary_logloss: 0.160715	valid_1's auc: 0.962856	valid_1's binary_logloss: 0.213812
[1135]	training's auc: 0.981615	training's binary_logloss: 0.160667	valid_1's auc: 0.962853	valid_1's binary_logloss: 0.213822
[1136]	training's auc: 0.981631	training's binary_logloss: 0.16062	valid_1's auc: 0.962858	valid_1's binary_logl

[1194]	training's auc: 0.982438	training's binary_logloss: 0.158121	valid_1's auc: 0.962852	valid_1's binary_logloss: 0.213846
[1195]	training's auc: 0.982449	training's binary_logloss: 0.158083	valid_1's auc: 0.962851	valid_1's binary_logloss: 0.213849
[1196]	training's auc: 0.982463	training's binary_logloss: 0.15804	valid_1's auc: 0.962852	valid_1's binary_logloss: 0.213847
[1197]	training's auc: 0.982477	training's binary_logloss: 0.157995	valid_1's auc: 0.962853	valid_1's binary_logloss: 0.213846
[1198]	training's auc: 0.982492	training's binary_logloss: 0.157952	valid_1's auc: 0.962854	valid_1's binary_logloss: 0.213844
[1199]	training's auc: 0.982504	training's binary_logloss: 0.157914	valid_1's auc: 0.962852	valid_1's binary_logloss: 0.21385
[1200]	training's auc: 0.982517	training's binary_logloss: 0.157875	valid_1's auc: 0.962852	valid_1's binary_logloss: 0.21385
Validation Gini: 0.79710, AUC: 0.9629


In [48]:
sub = pd.read_csv('sample_submission.csv')
sub['prediction']=np.mean(gbm_test_preds, axis=0)
sub.to_csv('submission_combin.csv', index=False)